# PyTorch 基础 : 自动求导
深度学习的算法本质上是通过反向传播求导数，而PyTorch的autograd模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。

***从0.4起, Variable 正式合并入Tensor, Variable 本来实现的自动微分功能，Tensor就能支持。读者还是可以使用Variable(tensor), 但是这个操作其实什么都没做。***

所以，以后的代码建议直接使用Tensor，因为官方文档中已经将Variable设置成过期模块

要想使得Tensor使用autograd功能，只需要设置tensor.requries_grad=True

In [1]:
# 首先要引入相关的包
import torch
#打印一下版本
torch.__version__

'1.0.1.post2'

在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动的求导，PyTorch回记录该张量的每一步操作历史并自动计算

In [2]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.4571, 0.8971, 0.1390, 0.8002, 0.3002],
        [0.9023, 0.6908, 0.1506, 0.4891, 0.5902],
        [0.3956, 0.3322, 0.2002, 0.7363, 0.9438],
        [0.3386, 0.5671, 0.6183, 0.4233, 0.2240],
        [0.5323, 0.9831, 0.2592, 0.4782, 0.3823]], requires_grad=True)

In [3]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.1946, 0.5275, 0.0394, 0.9823, 0.7158],
        [0.4545, 0.7747, 0.7586, 0.2620, 0.2254],
        [0.1506, 0.6088, 0.6685, 0.3700, 0.6479],
        [0.2825, 0.9235, 0.8427, 0.7171, 0.6873],
        [0.5856, 0.9442, 0.1404, 0.9213, 0.6899]], requires_grad=True)

我们看到 该张量的grad_fn已经被赋予了一个新的函数，下面我们来调用反向传播函数，计算其梯度

In [4]:
z=torch.sum(x+y)
z

tensor(26.9458, grad_fn=<SumBackward0>)

## 简单的自动求导

In [5]:
z.backward()

In [6]:
#看一下x和y的梯度
print(x.grad,y.grad)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


## 复杂的自动求导

In [7]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
z

tensor([[0.7221, 0.1057, 0.9276, 0.9994, 0.7355],
        [0.3914, 0.0511, 1.8594, 0.0732, 1.1167],
        [1.0665, 0.2872, 0.6940, 0.7090, 0.7666],
        [0.3997, 0.4208, 0.8164, 0.3546, 0.8069],
        [0.1061, 0.2634, 0.5090, 1.5252, 0.9312]], grad_fn=<AddBackward0>)

In [8]:
#我们的返回值不是一个scalar，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like函数根据x生成一个张量
z.backward(torch.ones_like(x))
print(x.grad)

tensor([[1.6988, 0.5991, 1.9076, 1.4065, 0.4551],
        [0.5436, 0.0545, 1.8707, 0.5061, 0.7158],
        [1.3221, 0.5068, 0.8729, 1.6788, 1.7406],
        [1.2645, 0.4477, 1.8063, 1.1824, 1.7786],
        [0.5552, 1.0176, 1.1920, 1.7473, 1.9300]])


我们可以使用`with torch.no_grad()`禁止已经设置requires_grad=True的向量进行自动求导，这个方法在测试集测试准确率的时候回经常用到，例如

In [9]:
with torch.no_grad():
    print((x +y*2).requires_grad)

False
